In [4]:
import pandas as pd
df1 = pd.read_csv('./data/tmdb_5000_credits.csv')
df2 = pd.read_csv('./data/tmdb_5000_movies.csv')

In [5]:
df1.columns=['id', 'title', 'cast', 'crew']

In [6]:
df_temp = df1[['id', 'cast', 'crew']]

In [7]:
df2 = df2.merge(df_temp, on = 'id')

In [8]:
df2.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
import pickle

In [10]:
temp = df2[['id', 'title']]
temp

,id,title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter
...,...,...
4798,9367,El Mariachi
4799,72766,Newlyweds
4800,231617,"Signed, Sealed, Delivered"
4801,126186,Shanghai Calling


In [11]:
pickle.dump(temp, open('./data/movies.pickle', 'wb'))

In [12]:
movies = pickle.load(open('./data/movies.pickle', 'rb'))
movies.head(10)

,id,title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter
5,559,Spider-Man 3
6,38757,Tangled
7,99861,Avengers: Age of Ultron
8,767,Harry Potter and the Half-Blood Prince
9,209112,Batman v Superman: Dawn of Justice


In [13]:
df2.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [14]:
df2['overview'].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [15]:
df2.shape

(4803, 22)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [17]:
df2['overview'] = df2['overview'].fillna('')

In [18]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])

In [19]:
tfidf_matrix.shape

(4803, 20978)

In [20]:
# 당어 유사도 구하기

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [21]:
cosine_sim.shape

(4803, 4803)

In [22]:
df2['title'].head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: title, dtype: object

In [24]:
filter = df2['title'] == 'The Dark Knight Rises'
index = df2[filter].index[0]
index

3

In [27]:
sim_scores = cosine_sim[index]
sim_scores

array([0.02499512, 0.        , 0.        , ..., 0.03386366, 0.04275232,
       0.02269198])

In [28]:
sim_scores = list(enumerate(sim_scores))
sim_scores

[(0, 0.0249951158376727),
 (1, 0.0),
 (2, 0.0),
 (3, 1.0),
 (4, 0.010433403719159354),
 (5, 0.0051446018158107934),
 (6, 0.01260063243546246),
 (7, 0.026954270578912674),
 (8, 0.02065221688538951),
 (9, 0.1337400906655523),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.004071333922512107),
 (16, 0.021121093874993183),
 (17, 0.0),
 (18, 0.006768893195007471),
 (19, 0.010765175685064708),
 (20, 0.007178266390761152),
 (21, 0.033380775071488206),
 (22, 0.0),
 (23, 0.0),
 (24, 0.019238168304196293),
 (25, 0.017013388161368177),
 (26, 0.018845673291717265),
 (27, 0.0),
 (28, 0.008351231142809445),
 (29, 0.02582248131068942),
 (30, 0.017549884233472455),
 (31, 0.029947529512076927),
 (32, 0.021708602796233892),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.028418448812136628),
 (37, 0.0),
 (38, 0.030631547964966054),
 (39, 0.01088522587631032),
 (40, 0.0),
 (41, 0.021109848007927),
 (42, 0.008087287409335878),
 (43, 0.0),
 (44, 0.0),
 (45, 0.008667206203132952),
 (46, 0.0

In [31]:
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
sim_scores[1:11]

[(65, 0.30151176591665485),
 (299, 0.29857045255396825),
 (428, 0.2878505467001694),
 (1359, 0.264460923827995),
 (3854, 0.18545003006561456),
 (119, 0.16799626199850706),
 (2507, 0.16682891043358278),
 (9, 0.1337400906655523),
 (1181, 0.13219702138476813),
 (210, 0.13045537014449818)]

In [38]:
df2['title'].iloc[299]

'Batman Forever'

In [39]:
sim_movies = [i[0] for i in sim_scores[1 : 11]]
sim_movies

[65, 299, 428, 1359, 3854, 119, 2507, 9, 1181, 210]

In [41]:
df2['title'].iloc[sim_movies]

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [42]:
pickle.dump(cosine_sim, open('./data/cosine_sim.pickle', 'wb'))